# Creating and Valuing a CDS Contract

In [1]:
import sys
sys.path.append("..")
sys.path.append("..\\..")

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from financepy.finutils.FinDate import FinDate
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinMath import ONE_MILLION
from financepy.products.libor.FinLiborDeposit import FinLiborDeposit
from financepy.products.libor.FinLiborSwap import FinLiborSwap
from financepy.market.curves.FinLiborCurve import FinLiborCurve
from financepy.products.credit.FinCDS import FinCDS
from financepy.market.curves.FinCDSCurve import FinCDSCurve

## Creating a CDS Contract

In [4]:
tradeDate = FinDate(2019, 8, 9)
effectiveDate = tradeDate
maturityDate = FinDate(2029, 6, 20)
cdsCoupon = 0.0150
notional = ONE_MILLION
longProtection = False
tradeDate = FinDate(2019, 8, 9)

In [14]:
cdsContract = FinCDS(effectiveDate, maturityDate, cdsCoupon, notional, longProtection)

In [15]:
cdsContract.print(settlementDate)

STEPINDATE:  FRI 9 AUG 2019
MATURITY:  WED 20 JUN 2029
NOTIONAL: 1000000
RUNNING COUPON:  150.0 bp
DAYCOUNT:  FinDayCountTypes.ACT_360
FREQUENCY:  FinFrequencyTypes.QUARTERLY
CALENDAR:  FinCalendarTypes.WEEKEND
BUSDAYRULE:  FinDayAdjustTypes.FOLLOWING
DATEGENRULE:  FinDateGenRuleTypes.BACKWARD
ACCRUED DAYS: 50
PAYMENT_DATE     YEAR_FRAC       FLOW
FRI 20 SEP 2019   0.255556      3833.33
FRI 20 DEC 2019   0.252778      3791.67
FRI 20 MAR 2020   0.252778      3791.67
MON 22 JUN 2020   0.261111      3916.67
MON 21 SEP 2020   0.252778      3791.67
MON 21 DEC 2020   0.252778      3791.67
MON 22 MAR 2021   0.252778      3791.67
MON 21 JUN 2021   0.252778      3791.67
MON 20 SEP 2021   0.252778      3791.67
MON 20 DEC 2021   0.252778      3791.67
MON 21 MAR 2022   0.252778      3791.67
MON 20 JUN 2022   0.252778      3791.67
TUE 20 SEP 2022   0.255556      3833.33
TUE 20 DEC 2022   0.252778      3791.67
MON 20 MAR 2023   0.250000      3750.00
TUE 20 JUN 2023   0.255556      3833.33
WED 20 SEP

## Build Libor Curve

In [16]:
dcType = FinDayCountTypes.ACT_360
depo1 = FinLiborDeposit(settlementDate, "1M", 0.022009, dcType)
depo2 = FinLiborDeposit(settlementDate, "2M", 0.022138, dcType)
depo3 = FinLiborDeposit(settlementDate, "3M", 0.021810, dcType)
depo4 = FinLiborDeposit(settlementDate, "6M", 0.020503, dcType)
depo5 = FinLiborDeposit(settlementDate, "12M", 0.019930, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swap1 = FinLiborSwap(settlementDate,"2Y",0.015910,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"3Y",0.014990,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"4Y",0.014725,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"5Y",0.014640,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"6Y",0.014800,fixedFreq,dcType)
swap6 = FinLiborSwap(settlementDate,"7Y",0.014995,fixedFreq,dcType)
swap7 = FinLiborSwap(settlementDate,"8Y",0.015180,fixedFreq,dcType)
swap8 = FinLiborSwap(settlementDate,"9Y",0.015610,fixedFreq,dcType)
swap9 = FinLiborSwap(settlementDate,"10Y",0.0159880,fixedFreq,dcType)
swap10 = FinLiborSwap(settlementDate,"12Y",0.016430,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

liborCurve = FinLiborCurve("USD_LIBOR", settlementDate, depos, [], swaps)

# Build a CDS Curve

In [17]:
cds1 = FinCDS(settlementDate, "1Y", 0.0200)
cds2 = FinCDS(settlementDate, "2Y", 0.0220)
cds3 = FinCDS(settlementDate, "3Y", 0.0250)
cds4 = FinCDS(settlementDate, "4Y", 0.0275)
cds5 = FinCDS(settlementDate, "5Y", 0.0290)
cds6 = FinCDS(settlementDate, "7Y", 0.0300)
cds7 = FinCDS(settlementDate, "10Y", 0.0310)
cds8 = FinCDS(settlementDate, "15Y", 0.0315)

In [18]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [19]:
recoveryRate = 0.40

In [20]:
issuerCurve = FinCDSCurve(settlementDate, cdss, liborCurve, recoveryRate)

In [21]:
print(issuerCurve._times)

[ 0.          1.00207534  2.00141276  3.00075019  4.00008761  5.00216295
  7.0008378  10.00158799 15.00101303]


In [22]:
print(issuerCurve._values)

[1.         0.9666732  0.92812751 0.8800167  0.82810907 0.77902806
 0.69646957 0.58522114 0.44200455]


# Valuation

In [23]:
spd = cdsContract.parSpread(settlementDate, issuerCurve, recoveryRate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  309.68697 bp


In [24]:
fullv, cleanv = cdsContract.value(settlementDate, issuerCurve, recoveryRate)
print("FULL VALUE  %12.2f"% fullv)
print("CLEAN VALUE %12.2f"% cleanv)

FULL VALUE    -115966.46
CLEAN VALUE   -118049.80


In [25]:
cleanp = cdsContract.cleanPrice(settlementDate, issuerCurve, recoveryRate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE    88.195322


In [26]:
accruedDays = cdsContract.accruedDays()
print("ACCRUED_DAYS", accruedDays)

ACCRUED_DAYS 50


In [27]:
accruedInterest = cdsContract.accruedInterest()
print("ACCRUED_COUPON", accruedInterest)

ACCRUED_COUPON 2083.3333333333335


In [28]:
protPV = cdsContract.protectionLegPV(settlementDate, issuerCurve, recoveryRate)
print("PROTECTION_PV", protPV)

PROTECTION_PV 228938.4289157856


In [29]:
premPV = cdsContract.premiumLegPV(settlementDate, issuerCurve, recoveryRate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 112971.96427930315


In [30]:
fullRPV01, cleanRPV01 = cdsContract.riskyPV01(settlementDate, issuerCurve)
print("FULL_RPV01", fullRPV01)
print("CLEAN_RPV01", cleanRPV01)

FULL_RPV01 7.531464285286877
CLEAN_RPV01 7.392575396397988
